In [22]:
!pip install requests beautifulsoup4 lxml
!pip install --upgrade selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 68.3 MB/s eta 0:00:000m eta 0:00:010:00:01
  Attempting uninstall: selenium
    Found existing installation: selenium 4.22.0
    Uninstalling selenium-4.22.0:
      Successfully uninstalled selenium-4.22.0


In [18]:
import requests
from bs4 import BeautifulSoup
import os

# Base URL of the GitHub repository
base_url = "https://github.com/redcanaryco/atomic-red-team/tree/master/atomics"

# Directory to save the combined text file
output_dir = "atomic_files"
os.makedirs(output_dir, exist_ok=True)
output_file_path = os.path.join(output_dir, "combined_atomic_files.txt")

def fetch_file_urls(directory_url):
    """Fetch URLs of .md files from a given directory URL."""
    print(f"Fetching {directory_url}")
    response = requests.get(directory_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'lxml')

    file_urls = []
    for link in soup.find_all('a', class_='js-navigation-open'):
        href = link.get('href')
        if href and href.endswith('.md'):
            file_url = f"https://github.com{href.replace('/blob', '/raw')}"
            file_urls.append(file_url)
            print(f"Found file: {file_url}")
    
    return file_urls

def download_file(file_url):
    """Download a file from URL and return its content."""
    print(f"Downloading {file_url}")
    response = requests.get(file_url)
    response.raise_for_status()
    return response.text

def scrape_specific_directories(base_url, start, end):
    """Scrape directories within a specific range and save content to a single text file."""
    print(f"Scraping directories from {start} to {end}")

    # Fetch the base directory page
    response = requests.get(base_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'lxml')

    # Debug output for inspection
    print("Page content:")
    print(soup.prettify()[:1000])  # Print first 1000 characters for debugging

    # Find all directory links
    dir_links = soup.find_all('a', class_='js-navigation-open')
    print(f"Found {len(dir_links)} links on the base directory page.")
    
    # Extract and print all hrefs for debugging
    for link in dir_links:
        href = link.get('href')
        print(f"Link href: {href}")
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for link in dir_links:
            href = link.get('href')
            if href and not href.endswith('.md') and href.count('/') == 4:  # Ensure it's a directory link
                directory_name = href.split('/')[-1]
                if directory_name.startswith('T') and directory_name[1:].replace('.', '').isdigit():
                    # Extract directory number and check range
                    dir_number = directory_name[1:].replace('.', '')
                    if start <= dir_number <= end:
                        directory_url = f"https://github.com{href.replace('/tree', '/blame')}"
                        print(f"Found directory: {directory_url}")
                        # Fetch the .md files in the directory
                        file_urls = fetch_file_urls(directory_url)
                        for file_url in file_urls:
                            file_name = file_url.split('/')[-1]
                            content = download_file(file_url)
                            if content:
                                # Write the content to the output file
                                output_file.write(f"--- Start of {file_name} ---\n")
                                output_file.write(content)
                                output_file.write(f"\n--- End of {file_name} ---\n\n")
                                print(f"Appended content of {file_name}")

# Define the range of directories to scrape
start_range = '1001'
end_range = '1654'

# Start scraping
scrape_specific_directories(base_url, start_range, end_range)


Scraping directories from 1001 to 1654
Page content:
<!DOCTYPE html>
<html data-a11y-animated-images="system" data-a11y-link-underlines="true" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
  <link href="https://avatars.githubusercontent.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-efd2f2257c96.css" media="all" rel="stylesheet"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-6b1e37da2254.css" media="all" rel="stylesheet"/>
  <link crossorigin="anonymous

In [36]:
import requests
from bs4 import BeautifulSoup
import os
import re

# Base URL of the GitHub repository
base_url = "https://github.com/redcanaryco/atomic-red-team/tree/master/atomics"

# Directory to save the downloaded .md files
output_dir = "atomic_files2"
os.makedirs(output_dir, exist_ok=True)

def fetch_directory_links(url):
    """Fetch directory links from a given URL."""
    print(f"Fetching {url}")
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract links that point to directories
    dir_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if href.startswith('/redcanaryco/atomic-red-team/tree/master/atomics/'):
            full_url = f"https://github.com{href}"
            dir_links.append(full_url)
            print(f"Found directory link: {full_url}")
    
    return dir_links

def fetch_file_urls(directory_url):
    """Fetch URLs of .md files from a given directory URL."""
    print(f"Fetching files from {directory_url}")
    response = requests.get(directory_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    file_urls = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if href.endswith('.md'):
            file_url = f"https://github.com{href.replace('/blob', '/raw')}"
            file_urls.append(file_url)
            print(f"Found file: {file_url}")
    
    return file_urls

def download_file(file_url):
    """Download a file from URL and return its content."""
    print(f"Downloading {file_url}")
    response = requests.get(file_url)
    response.raise_for_status()
    return response.text

def extract_directory_number(name):
    """Extract directory number from the directory name."""
    match = re.match(r'T(\d+(\.\d+)*)', name)
    return match.group(1) if match else None

def scrape_directories(base_url, start, end):
    """Scrape directories and save each .md file as a separate text file."""
    print(f"Scraping directories from {start} to {end}")
    
    # Fetch the base directory page
    directory_links = fetch_directory_links(base_url)
    
    for link in directory_links:
        # Fetch the directory page and its subdirectories or files
        subdirectory_links = fetch_directory_links(link)
        
        # Combine directory links (subdirectories) and current directory
        all_directory_links = [link] + subdirectory_links
        
        for dir_link in all_directory_links:
            # Extract directory name from the URL
            dir_name = dir_link.split('/')[-1]
            dir_number = extract_directory_number(dir_name)
            
            if dir_number and start <= dir_number <= end:
                file_urls = fetch_file_urls(dir_link)
                for file_url in file_urls:
                    file_name = file_url.split('/')[-1]
                    content = download_file(file_url)
                    if content:
                        file_path = os.path.join(output_dir, file_name)
                        # Save each .md file as a separate text file
                        with open(file_path, 'w', encoding='utf-8') as file:
                            file.write(content)
                        print(f"Saved {file_name} to {file_path}")

# Define the range of directories to scrape
start_range = '1001'
end_range = '1654'

# Start scraping
scrape_directories(base_url, start_range, end_range)


Scraping directories from 1001 to 1654
Fetching https://github.com/redcanaryco/atomic-red-team/tree/master/atomics
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/Indexes
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/Indexes
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1001.002
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1001.002
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1003.001
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1003.001
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1003.002
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1003.002
Found directory link: https://github.com/redcanaryco/atomic-red-team/tree/master/atomics/T1003.00